In [2]:
import pandas as pd

file_name = "500_Reddit_users_posts_labels.csv"

df = pd.read_csv(file_name)

df.head()

,User,Post,Label
0,user-0,"['Its not a viable option, and youll be leavin...",Supportive
1,user-1,['It can be hard to appreciate the notion that...,Ideation
2,user-2,"['Hi, so last night i was sitting on the ledge...",Behavior
3,user-3,['I tried to kill my self once and failed badl...,Attempt
4,user-4,['Hi NEM3030. What sorts of things do you enjo...,Ideation


In [ ]:
label_counts = df['Label'].value_counts()
print(label_counts)

Label
Ideation      171
Supportive    108
Indicator      99
Behavior       77
Attempt        45
Name: count, dtype: int64


In [3]:
balanced_data = []
copies = df[df['Label'] == 'Attempt'].sample(n=125, replace=True)
balanced_data.append(copies)

In [4]:
copies = df[df['Label'] == 'Behavior'].sample(n=100, replace=True)
balanced_data.append(copies)

In [5]:
copies = df[df['Label'] == 'Indicator'].sample(n=70, replace=True)
balanced_data.append(copies)

In [6]:
copies = df[df['Label'] == 'Supportive'].sample(n=60, replace=True)
balanced_data.append(copies)

In [7]:
balanced_df = pd.concat([df] + balanced_data)

In [8]:
label_counts = balanced_df['Label'].value_counts()
print(label_counts)

Label
Behavior      177
Ideation      171
Attempt       170
Indicator     169
Supportive    168
Name: count, dtype: int64


In [8]:
clean_zero = balanced_df.drop('User', axis=1)

In [9]:
encoding_mapping = {'Behavior': 1, 'Ideation': 0, 'Attempt': 2, 'Indicator': -1, 'Supportive': -2}
clean_zero['Encoded_Labels'] = clean_zero['Label'].replace(encoding_mapping)

In [10]:
clean_data = clean_zero.drop('Label', axis=1)
clean_data

,Post,Encoded_Labels
0,"['Its not a viable option, and youll be leavin...",-2
1,['It can be hard to appreciate the notion that...,0
2,"['Hi, so last night i was sitting on the ledge...",1
3,['I tried to kill my self once and failed badl...,2
4,['Hi NEM3030. What sorts of things do you enjo...,0
...,...,...
468,['You obviously care for your mother quite dee...,-2
494,['Be her better half. You have to put yourself...,-2
20,['I hope you both can read this. Reading both ...,-2
403,['Friendships are complicated. Close friendsh...,-2


In [11]:
import numpy as np

data_array = clean_data.values

In [12]:
def split_data(data_array, train_size=0.7, val_size=0.1, test_size=0.2):
    # Ensure the sizes sum up to 1
    assert (train_size + val_size + test_size) == 1, "The sizes must sum up to 1."

    # Shuffle the array
    np.random.shuffle(data_array)

    # Calculate split indices
    total_samples = len(data_array)
    train_end = int(total_samples * train_size)
    val_end = train_end + int(total_samples * val_size)

    # Split the dataset
    train_data = data_array[:train_end]
    val_data = data_array[train_end:val_end]
    test_data = data_array[val_end:]

    return train_data, val_data, test_data

train, val, test = split_data(data_array)

In [15]:
test[8]

array(['[\'Your dad loves you. Call him and tell him youre having a hard time and need some help. If my son did this, I wouldnt feel the slightest bit of disappointment, Id be on that shit Hyperactive behavior a ninja paratrooper and wed fix that shit. You need him, but he needs you too.\', \'I think so, yes.\', \'Whatever you do, get the fuck out of that school. Its killing your soul and that is a wound that is tough to heal. I went to religious school my whole life, was basically "indoctrinated"  Told how to think, what to believe, when to sit, stand, pray. I could go on and on. This is the time on your life for you to figure out who you are, not who someone else wants you to be. Tell your parents you love and respect them, but you need to do what is right for you. Try telling them your gay first to soften the blow, then say "just kidding, Im not religious"Let us know how it goes\', \'Dude, I do the same thing. It makes reading comprehension much easier. Im in my late 30s, so there w

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False)

# Fit and transform the training data to compute TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(train[:,0])

# Transform the validation and test data (do not fit, to ensure consistency with the training data vocabulary)
X_val_tfidf = tfidf_vectorizer.transform(val[:,0])
X_test_tfidf = tfidf_vectorizer.transform(test[:,0])

In [24]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')
X_train_embeddings = model.encode(train[:,0], batch_size=32, show_progress_bar=True)
X_val_embeddings = model.encode(val[:,0], batch_size=32, show_progress_bar=True)
X_test_embeddings = model.encode(test[:,0], batch_size=32, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/19 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [26]:
y_train = train[:, 1].astype(int)
y_val = val[:,1].astype(int)
y_test = test[:,1].astype(int)

In [27]:
y_train

array([2, 4, 3, 3, 0, 0, 4, 0, 2, 4, 3, 3, 4, 2, 3, 1, 3, 1, 4, 4, 0, 3,
       4, 3, 1, 1, 2, 4, 4, 3, 3, 0, 0, 4, 1, 1, 1, 4, 3, 4, 3, 4, 1, 3,
       1, 0, 3, 1, 1, 3, 2, 4, 0, 1, 0, 2, 1, 0, 4, 0, 1, 4, 4, 3, 1, 4,
       1, 3, 4, 0, 0, 0, 3, 2, 4, 4, 2, 1, 3, 2, 3, 4, 4, 0, 3, 3, 3, 3,
       2, 4, 4, 0, 3, 1, 1, 1, 1, 2, 4, 1, 2, 2, 4, 0, 4, 4, 4, 2, 1, 4,
       3, 1, 0, 2, 4, 2, 2, 1, 1, 0, 4, 4, 4, 3, 3, 0, 3, 2, 0, 2, 2, 4,
       3, 2, 4, 4, 3, 3, 0, 0, 0, 4, 1, 4, 0, 2, 4, 4, 2, 2, 3, 1, 3, 0,
       4, 2, 2, 4, 1, 1, 3, 0, 0, 2, 0, 4, 2, 3, 4, 4, 2, 3, 2, 3, 4, 2,
       1, 1, 0, 2, 1, 4, 3, 0, 1, 1, 4, 2, 4, 1, 3, 0, 1, 4, 4, 4, 4, 2,
       0, 3, 0, 4, 4, 3, 1, 1, 0, 0, 2, 0, 4, 0, 4, 1, 2, 3, 1, 0, 0, 3,
       3, 1, 4, 4, 0, 4, 3, 0, 1, 0, 0, 2, 3, 4, 3, 0, 4, 1, 0, 1, 0, 1,
       4, 3, 3, 3, 3, 1, 1, 2, 3, 2, 4, 3, 1, 2, 1, 0, 2, 0, 2, 2, 4, 3,
       1, 1, 3, 3, 1, 0, 3, 0, 0, 0, 3, 3, 2, 3, 0, 0, 1, 0, 0, 0, 1, 0,
       2, 0, 2, 1, 1, 1, 2, 1, 2, 0, 3, 4, 4, 2, 2,

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid


# Define different configurations for each model
models_configurations = {
    'Logistic Regression': {'C': [0.001, 0.1, 1.0], 'penalty':['l1','l2',None], 'tol':[0.0001, 0.001, 0.01], 'solver':['saga']},
    'SVM': {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']},
    'Naive Bayes': {'alpha': [0.1, 1.0, 10.0]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'criterion':['gini', 'entropy', 'log_loss'], 'random_state':[0]}
}

validation_f1_scores = {model: [] for model in models_configurations}

# Train each model using different configurations
for model_name, config in models_configurations.items():
    if model_name == 'Logistic Regression':
        model = LogisticRegression(max_iter=4000)
        print("Model is linreg")
    elif model_name == 'SVM':
        model = SVC()
        print("Model is SVM")
    elif model_name == 'Naive Bayes':
        model = MultinomialNB()
        print("Model if MNB")
    elif model_name == 'Random Forest':
        model = RandomForestClassifier()
        print("Model is RF")
    else:
        raise ValueError(f"Invalid model name: {model_name}")

    # Train the model with different configurations
    for params in ParameterGrid(config):
        model.set_params(**params)
        model.fit(X_train_tfidf, y_train)

        # Evaluate on validation set
        y_pred = model.predict(X_val_tfidf)
        f1 = f1_score(y_val, y_pred, average='weighted')  #

        # Store validation F1 score for this configuration
        validation_f1_scores[model_name].append((params, f1))

# Choose the best configuration based on validation set performance
best_model_configurations = {}
for model_name, f1_scores in validation_f1_scores.items():
    best_config = max(f1_scores, key=lambda x: x[1])
    best_model_configurations[model_name] = best_config


Model is linreg


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Model is SVM
Model if MNB
Model is RF


In [29]:
print(best_model_configurations)

{'Logistic Regression': ({'C': 0.1, 'penalty': None, 'solver': 'saga', 'tol': 0.001}, 0.6267286502580619), 'SVM': ({'C': 10.0, 'kernel': 'rbf'}, 0.6814593430510385), 'Naive Bayes': ({'alpha': 0.1}, 0.6503077738371856), 'Random Forest': ({'criterion': 'entropy', 'n_estimators': 200, 'random_state': 0}, 0.7095031696889281)}


In [35]:
best_models = {}

for model_name, (best_config, _) in best_model_configurations.items():
    if model_name == 'Logistic Regression':
        model = LogisticRegression(max_iter=4000)
    elif model_name == 'SVM':
        model = SVC()
    elif model_name == 'Naive Bayes':
        model = MultinomialNB()
    elif model_name == 'Random Forest':
        model = RandomForestClassifier()
    else:
        raise ValueError(f"Invalid model name: {model_name}")


    model.set_params(**best_config)


    model.fit(X_train_tfidf, y_train)


    y_pred_test = model.predict(X_test_tfidf)


    test_f1 = f1_score(y_test, y_pred_test, average='weighted')

    true_labels = y_test.tolist()
    predicted_labels = y_pred_test.tolist()

    best_models[model_name] = {'model': model, 'test_f1': test_f1, 'true_labels': true_labels, 'predicted_labels': predicted_labels}


# Print the F1 scores of the best models on the test set
for model_name, model_info in best_models.items():
    print(f"{model_name}: F1 score on test set = {model_info['test_f1']}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Logistic Regression: F1 score on test set = 0.7415972308585168
SVM: F1 score on test set = 0.765251996564961
Naive Bayes: F1 score on test set = 0.7250394842854536
Random Forest: F1 score on test set = 0.7911178887869855


In [36]:
for model_name, model_info in best_models.items():
    true_labels = model_info['true_labels']
    predicted_labels = model_info['predicted_labels']
    incorrect_indices = [i for i, (true, pred) in enumerate(zip(true_labels, predicted_labels)) if true != pred]
    print(f"Incorrect predictions for {model_name}: {len(incorrect_indices)}")
    for idx in incorrect_indices:
        print(f"Index {idx}: True label = {true_labels[idx]}, Predicted label = {predicted_labels[idx]}")

Incorrect predictions for Logistic Regression: 43
Index 4: True label = 0, Predicted label = 1
Index 8: True label = 0, Predicted label = 4
Index 9: True label = 0, Predicted label = 1
Index 16: True label = 2, Predicted label = 1
Index 24: True label = 1, Predicted label = 2
Index 25: True label = 2, Predicted label = 1
Index 27: True label = 2, Predicted label = 3
Index 33: True label = 3, Predicted label = 4
Index 40: True label = 2, Predicted label = 3
Index 47: True label = 2, Predicted label = 3
Index 49: True label = 2, Predicted label = 1
Index 50: True label = 3, Predicted label = 1
Index 52: True label = 2, Predicted label = 1
Index 56: True label = 1, Predicted label = 0
Index 64: True label = 3, Predicted label = 0
Index 68: True label = 0, Predicted label = 2
Index 73: True label = 0, Predicted label = 1
Index 77: True label = 2, Predicted label = 1
Index 81: True label = 3, Predicted label = 2
Index 84: True label = 2, Predicted label = 3
Index 89: True label = 0, Predict

In [37]:
embeddings_validation_f1_scores = {model: [] for model in models_configurations}

# Train each model using different configurations
for model_name, config in models_configurations.items():
    if model_name == 'Logistic Regression':
        model = LogisticRegression(max_iter=4000)
        print("Model is linreg")
    elif model_name == 'SVM':
        model = SVC()
        print("Model is SVM")
    elif model_name == 'Naive Bayes':
        continue
        print("Model if MNB")
    elif model_name == 'Random Forest':
        model = RandomForestClassifier()
        print("Model is RF")
    else:
        raise ValueError(f"Invalid model name: {model_name}")

    # Train the model with different configurations
    for params in ParameterGrid(config):
        model.set_params(**params)
        model.fit(X_train_embeddings, y_train)

        # Evaluate on validation set
        y_pred = model.predict(X_val_embeddings)
        f1 = f1_score(y_val, y_pred, average='weighted')  #

        # Store validation F1 score for this configuration
        embeddings_validation_f1_scores[model_name].append((params, f1))



Model is linreg


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Model is SVM
Model is RF


In [38]:
print(embeddings_validation_f1_scores)

{'Logistic Regression': [({'C': 0.001, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.0001}, 0.06666666666666667), ({'C': 0.001, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.001}, 0.06666666666666667), ({'C': 0.001, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.01}, 0.052941176470588235), ({'C': 0.001, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.0001}, 0.052941176470588235), ({'C': 0.001, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.001}, 0.052941176470588235), ({'C': 0.001, 'penalty': 'l2', 'solver': 'saga', 'tol': 0.01}, 0.052941176470588235), ({'C': 0.001, 'penalty': None, 'solver': 'saga', 'tol': 0.0001}, 0.6530358089181619), ({'C': 0.001, 'penalty': None, 'solver': 'saga', 'tol': 0.001}, 0.6554533351085077), ({'C': 0.001, 'penalty': None, 'solver': 'saga', 'tol': 0.01}, 0.6294245550282702), ({'C': 0.1, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.0001}, 0.052941176470588235), ({'C': 0.1, 'penalty': 'l1', 'solver': 'saga', 'tol': 0.001}, 0.10643210555250139), ({'C': 0.1, 'penalty': 'l1', '

In [39]:
# Choose the best configuration based on validation set performance
best_model_configurations = {}
for model_name, f1_scores in embeddings_validation_f1_scores.items():
    if model_name =='Naive Bayes':
      continue
    best_config = max(f1_scores, key=lambda x: x[1])
    best_model_configurations[model_name] = best_config

In [40]:
print(best_model_configurations)

{'Logistic Regression': ({'C': 0.001, 'penalty': None, 'solver': 'saga', 'tol': 0.001}, 0.6554533351085077), 'SVM': ({'C': 10.0, 'kernel': 'linear'}, 0.6993160597572362), 'Random Forest': ({'criterion': 'gini', 'n_estimators': 100, 'random_state': 0}, 0.7026709507332345)}


In [41]:
best_models = {}

for model_name, (best_config, _) in best_model_configurations.items():
    if model_name == 'Logistic Regression':
        model = LogisticRegression(max_iter=4000)
    elif model_name == 'SVM':
        model = SVC()
    elif model_name == 'Naive Bayes':
        continue
    elif model_name == 'Random Forest':
        model = RandomForestClassifier()
    else:
        raise ValueError(f"Invalid model name: {model_name}")


    model.set_params(**best_config)


    model.fit(X_train_embeddings, y_train)


    y_pred_test = model.predict(X_test_embeddings)


    test_f1 = f1_score(y_test, y_pred_test, average='weighted')


    true_labels = y_test.tolist()
    predicted_labels = y_pred_test.tolist()

    best_models[model_name] = {'model': model, 'test_f1': test_f1, 'true_labels': true_labels, 'predicted_labels': predicted_labels}


# Print the F1 scores of the best models on the test set
for model_name, model_info in best_models.items():
    print(f"{model_name}: F1 score on test set = {model_info['test_f1']}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Logistic Regression: F1 score on test set = 0.6838064220103196
SVM: F1 score on test set = 0.7003378988484923
Random Forest: F1 score on test set = 0.7436322730838524


In [42]:
for model_name, model_info in best_models.items():
    true_labels = model_info['true_labels']
    predicted_labels = model_info['predicted_labels']
    incorrect_indices = [i for i, (true, pred) in enumerate(zip(true_labels, predicted_labels)) if true != pred]
    print(f"Incorrect predictions for {model_name}: {len(incorrect_indices)}")
    for idx in incorrect_indices:
        print(f"Index {idx}: True label = {true_labels[idx]}, Predicted label = {predicted_labels[idx]}")

Incorrect predictions for Logistic Regression: 53
Index 1: True label = 2, Predicted label = 3
Index 3: True label = 1, Predicted label = 0
Index 4: True label = 0, Predicted label = 1
Index 5: True label = 2, Predicted label = 1
Index 9: True label = 0, Predicted label = 1
Index 16: True label = 2, Predicted label = 1
Index 23: True label = 2, Predicted label = 1
Index 24: True label = 1, Predicted label = 2
Index 27: True label = 2, Predicted label = 4
Index 33: True label = 3, Predicted label = 0
Index 49: True label = 2, Predicted label = 1
Index 50: True label = 3, Predicted label = 2
Index 52: True label = 2, Predicted label = 0
Index 59: True label = 0, Predicted label = 4
Index 63: True label = 0, Predicted label = 4
Index 64: True label = 3, Predicted label = 4
Index 66: True label = 2, Predicted label = 1
Index 68: True label = 0, Predicted label = 4
Index 73: True label = 0, Predicted label = 1
Index 79: True label = 2, Predicted label = 4
Index 81: True label = 3, Predicted

In [14]:
!export CUDA_LAUNCH_BLOCKING=1

In [15]:
train[:,1] = train[:,1] + 2
val[:,1] = val[:,1]+2

In [16]:
train[:,1]

array([2, 4, 3, 3, 0, 0, 4, 0, 2, 4, 3, 3, 4, 2, 3, 1, 3, 1, 4, 4, 0, 3,
       4, 3, 1, 1, 2, 4, 4, 3, 3, 0, 0, 4, 1, 1, 1, 4, 3, 4, 3, 4, 1, 3,
       1, 0, 3, 1, 1, 3, 2, 4, 0, 1, 0, 2, 1, 0, 4, 0, 1, 4, 4, 3, 1, 4,
       1, 3, 4, 0, 0, 0, 3, 2, 4, 4, 2, 1, 3, 2, 3, 4, 4, 0, 3, 3, 3, 3,
       2, 4, 4, 0, 3, 1, 1, 1, 1, 2, 4, 1, 2, 2, 4, 0, 4, 4, 4, 2, 1, 4,
       3, 1, 0, 2, 4, 2, 2, 1, 1, 0, 4, 4, 4, 3, 3, 0, 3, 2, 0, 2, 2, 4,
       3, 2, 4, 4, 3, 3, 0, 0, 0, 4, 1, 4, 0, 2, 4, 4, 2, 2, 3, 1, 3, 0,
       4, 2, 2, 4, 1, 1, 3, 0, 0, 2, 0, 4, 2, 3, 4, 4, 2, 3, 2, 3, 4, 2,
       1, 1, 0, 2, 1, 4, 3, 0, 1, 1, 4, 2, 4, 1, 3, 0, 1, 4, 4, 4, 4, 2,
       0, 3, 0, 4, 4, 3, 1, 1, 0, 0, 2, 0, 4, 0, 4, 1, 2, 3, 1, 0, 0, 3,
       3, 1, 4, 4, 0, 4, 3, 0, 1, 0, 0, 2, 3, 4, 3, 0, 4, 1, 0, 1, 0, 1,
       4, 3, 3, 3, 3, 1, 1, 2, 3, 2, 4, 3, 1, 2, 1, 0, 2, 0, 2, 2, 4, 3,
       1, 1, 3, 3, 1, 0, 3, 0, 0, 0, 3, 3, 2, 3, 0, 0, 1, 0, 0, 0, 1, 0,
       2, 0, 2, 1, 1, 1, 2, 1, 2, 0, 3, 4, 4, 2, 2,

In [18]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)

train_encodings = tokenizer(train[:,0].tolist(), truncation=True, padding=True, max_length=128)
train_labels = torch.tensor(train[:,1].tolist())

val_encodings = tokenizer(val[:,0].tolist(), truncation=True, padding=True, max_length=128)
val_labels = torch.tensor(val[:,1].tolist())

train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              train_labels)

val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']),
                            torch.tensor(val_encodings['attention_mask']),
                            val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    train_loader_with_progress = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for batch in train_loader:
        input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()


    model.eval()
    val_preds = []
    val_targets = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, axis=1)
            val_preds.extend(preds.cpu().numpy())
            val_targets.extend(labels.cpu().numpy())

    val_f1 = f1_score(val_targets, val_preds, average='weighted')
    print(f'Epoch {epoch + 1}, Validation F1 Score: {val_f1:.4f}')



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/50:   0%|          | 0/38 [00:00<?, ?it/s]
                                                   

Epoch 1, Validation F1 Score: 0.0673


Epoch 2, Validation F1 Score: 0.2892


Epoch 3/50:   0%|          | 0/38 [00:00<?, ?it/s]
                                                  

Epoch 3, Validation F1 Score: 0.3650


...

Epoch 47, Validation F1 Score: 0.6312


Epoch 48, Validation F1 Score: 0.5358


Epoch 49/50:   0%|          | 0/38 [00:00<?, ?it/s]
                                                   

Epoch 49, Validation F1 Score: 0.5620


Epoch 50, Validation F1 Score: 0.6429


In [21]:
test_encodings = tokenizer(test[:,0].tolist(), truncation=True, padding=True, max_length=128)
test_labels = torch.tensor(test[:,1].tolist())

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             test_labels)

test_loader = DataLoader(test_dataset, batch_size=16)

model.eval()
test_preds = []
test_targets = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = [tensor.to(device) for tensor in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, axis=1)
        test_preds.extend(preds.cpu().numpy())
        test_targets.extend(labels.cpu().numpy())

test_f1 = f1_score(test_targets, test_preds, average='weighted')
incorrect_indices = [i for i, (pred, target) in enumerate(zip(test_preds, test_targets)) if pred != target]
print(f'Test F1 Score: {test_f1:.4f}')

Test F1 Score: 0.7217


In [19]:
test[:,1] = test[:,1]+2

In [20]:
test[:,1]

array([1, 2, 1, 1, 0, 2, 3, 0, 0, 0, 0, 3, 0, 1, 1, 4, 2, 3, 1, 3, 1, 3,
       2, 2, 1, 2, 4, 2, 4, 4, 3, 1, 0, 3, 1, 3, 4, 4, 1, 1, 2, 3, 1, 4,
       0, 1, 4, 2, 4, 2, 3, 4, 2, 4, 1, 4, 1, 1, 2, 0, 3, 3, 0, 0, 3, 3,
       2, 2, 0, 3, 1, 4, 4, 0, 1, 4, 1, 2, 1, 2, 2, 3, 4, 1, 2, 3, 2, 1,
       1, 0, 0, 0, 0, 0, 4, 0, 0, 0, 2, 3, 3, 4, 3, 1, 1, 2, 1, 4, 0, 0,
       0, 2, 2, 1, 2, 4, 4, 0, 3, 2, 1, 2, 4, 1, 3, 1, 4, 0, 3, 1, 3, 1,
       4, 3, 4, 3, 3, 2, 3, 3, 1, 3, 4, 0, 1, 3, 0, 0, 3, 0, 3, 2, 4, 4,
       3, 0, 0, 2, 1, 3, 1, 3, 4, 4, 3, 4, 0, 4, 2, 0, 1, 1], dtype=object)

In [22]:
for idx in incorrect_indices:
    text, label = test[idx,0], test[idx,1]
    print("Example:", text)
    print("True Label:", label)
    print("Predicted Label:", test_preds[idx])
    print("="*50)

Example: ['Have you been on the same meds for the past 5 years? Therapists can offer you some good coping strategies go help you deal with the situation yourself during difficult times, but in my experience, some of them just plain suck. Finding a good one isnt the easiest thing. ', 'Let your child be the reason for you to keep going. Your child hasnt had a say into whether or not they would Hyperactive behavior to live. Please do not do this. Please try to push on. If you broke up with him it means you have respect for yourself, and part of you wants to live, and believes there is hope. There are so many things that can go wrong during pregnancy, so it is always a miracle when a baby makes it this far. Things will get better for you. ', 'Are you aware that there are other treatments? Therapy and meds are not the only option. ', 'Please pm me or something. Let me know that youre still alive. Give some sort of a response. Give an indication youre still alive. There are people out there 

In [43]:
len(incorrect_indices)

45

In [44]:
! pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=90fa36845ef62ec11ef5473c7684ae636fab35148cff2b1b26627cd69f4f6f87
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch